In [1]:
# imports
import os 
import pandas as pd 
import requests 

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [2]:
#Load API key from env variable 
api_key = os.environ["FOURSQUARE_API_KEY"]

In [3]:
def get_poi_fs(latitude, longitude, radius, api_key, categories):    
    url = f'https://api.foursquare.com/v3/places/search?ll={latitude},{longitude}&radius={radius}'

    # Creating dictionary for headers
    headers = {"Accept": "application/json"}
    headers['Authorization'] = api_key
    reponse = requests.get(url, headers=headers)
    reponse_data = reponse.json()
    
    return reponse_data['results']

In [4]:
# Testing above results
res = get_poi_fs(41.425364,2.185207,1000, api_key, categories=['restaurants','bars','pubs'])

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [5]:
# Parsing above response to get POI
## putting above response in a readable format
res[0]

{'fsq_id': '4e18b61d18a8166f738f1ed0',
 'categories': [{'id': 13026,
   'name': 'BBQ Joint',
   'short_name': 'BBQ',
   'plural_name': 'BBQ Joints',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bbqalt_',
    'suffix': '.png'}},
  {'id': 13338,
   'name': 'Seafood Restaurant',
   'short_name': 'Seafood',
   'plural_name': 'Seafood Restaurants',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/seafood_',
    'suffix': '.png'}},
  {'id': 13345,
   'name': 'Spanish Restaurant',
   'short_name': 'Spanish',
   'plural_name': 'Spanish Restaurants',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/spanish_',
    'suffix': '.png'}}],
 'chains': [],
 'closed_bucket': 'LikelyOpen',
 'distance': 215,
 'geocodes': {'main': {'latitude': 41.427162, 'longitude': 2.186739},
  'roof': {'latitude': 41.427162, 'longitude': 2.186739}},
 'link': '/v3/places/4e18b61d18a8166f738f1ed0',
 'location': {'address': 'Calle Torroella de Montgrí 7',
  'admin_r

Put your parsed results into a DataFrame

In [6]:
bike_stations_data1 = pd.read_csv('../data/bike_stations_barcelona.csv', index_col=0)

In [7]:
# Creating a DataFrame for Foursquare
foursquare_df = pd.DataFrame()

In [9]:
## Create an empty list 
dminlist = []
numlist = []

In [15]:
## Looping through bike_stations_data1 Dataframe to extract lat, long, and id of each row 
for index, row in bike_stations_data1.iterrows():
    poi_fs = get_poi_fs(row['latitude'], row['longitude'], 1000, api_key, categories=None)

    # put response into dataframe called "new_df_1"
    new_df_1 = pd.json_normalize(poi_fs, record_path=['categories'],meta=['fsq_id', 'name', 'distance', ['location', 'address']], meta_prefix='poi_', errors='ignore')

    # Adding column with the corresponding bike station id
    new_df_1['station_id'] = row['id']

    # Adding new_df_1 dataframe to yelp dataframe 
    foursquare_df = pd.concat([foursquare_df, new_df_1])

In [16]:
# Dropping unnecessary columns/ empty columns from foursquare dataframe
columns_to_drop_foursquare = ['short_name', 'plural_name', 'icon.prefix', 'icon.suffix']

foursquare_df = foursquare_df.drop(columns=columns_to_drop_foursquare)

# Renaming columns
foursquare_df = foursquare_df.rename(columns={"id": "category_id", "name": "category_name", 'poi_location.address': 'poi_address'})

In [17]:
# Reordering the columns above
foursquare_df = foursquare_df[['poi_fsq_id', 'poi_name', 'category_id', 'category_name', 'poi_distance', 'poi_address', 'station_id']]

In [18]:
## Printing shape and info of Dataframe after removing NAN/empty columns, and reording columns
print(foursquare_df.shape, '\n')
print(foursquare_df.info())

(8058, 7) 

<class 'pandas.core.frame.DataFrame'>
Index: 8058 entries, 0 to 15
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   poi_fsq_id     8058 non-null   object
 1   poi_name       8058 non-null   object
 2   category_id    8058 non-null   int64 
 3   category_name  8058 non-null   object
 4   poi_distance   8058 non-null   object
 5   poi_address    7709 non-null   object
 6   station_id     8058 non-null   object
dtypes: int64(1), object(6)
memory usage: 503.6+ KB
None


In [19]:
foursquare_df.head()

,poi_fsq_id,poi_name,category_id,category_name,poi_distance,poi_address,station_id
0,4e18b61d18a8166f738f1ed0,Mesón Pahi,13026,BBQ Joint,215,Calle Torroella de Montgrí 7,dedcd93fbe85671e73c79baf58bd7fed
1,4e18b61d18a8166f738f1ed0,Mesón Pahi,13338,Seafood Restaurant,215,Calle Torroella de Montgrí 7,dedcd93fbe85671e73c79baf58bd7fed
2,4e18b61d18a8166f738f1ed0,Mesón Pahi,13345,Spanish Restaurant,215,Calle Torroella de Montgrí 7,dedcd93fbe85671e73c79baf58bd7fed
3,590896ec5d891b372346f974,Cyclic Beer Farm,13029,Brewery,344,"Carrer de Coll, 37",dedcd93fbe85671e73c79baf58bd7fed
4,590896ec5d891b372346f974,Cyclic Beer Farm,13065,Restaurant,344,"Carrer de Coll, 37",dedcd93fbe85671e73c79baf58bd7fed


In [30]:
## Save above data to use within another notebook 
foursquare_df.to_csv('../data/bike_stations_barcelona_poi_fsq.csv')

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [31]:
import requests
import os
import pandas as pd 

In [32]:
## Loading environment variable for yelp using os.environment 
API_KEY = os.environ.get("yelp_api_key")

In [38]:
## Sending http request to Yelp API 
url = "https://api.yelp.com/v3/businesses/search?latitude=41.3874&longitude=2.1686&radius=1000&categories=bars%2C%20restaurant%2C%20pubs&sort_by=best_match&limit=20"

headers = {
    "accept": "application/json",
    "Authorization": "Bearer " + API_KEY
}

response = requests.get(url, headers=headers)
response_data = response.json()
print(response_data)

{'businesses': [{'id': 'Cn1geHmwrvKu1qQISgQKgA', 'alias': 'sensi-barcelona', 'name': 'Sensi', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/ndqinTlySy6slhGGeykyww/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/sensi-barcelona?adjust_creative=QyGIBXO3bewZz1kNqGAwCA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=QyGIBXO3bewZz1kNqGAwCA', 'review_count': 402, 'categories': [{'alias': 'tapasmallplates', 'title': 'Tapas/Small Plates'}, {'alias': 'tapas', 'title': 'Tapas Bars'}, {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}], 'rating': 4.5, 'coordinates': {'latitude': 41.38035, 'longitude': 2.17997}, 'transactions': [], 'price': '€€', 'location': {'address1': 'Carrer Ample, 26', 'address2': '', 'address3': '', 'city': 'Barcelona', 'zip_code': '08002', 'country': 'ES', 'state': 'B', 'display_address': ['Carrer Ample, 26', '08002 Barcelona', 'Spain']}, 'phone': '+34932956588', 'display_phone': '+34 932 95 65 88', 'distance': 1237.3374168059947}

In [39]:
## Putting above code in more readable format
response.json()

{'businesses': [{'id': 'Cn1geHmwrvKu1qQISgQKgA',
   'alias': 'sensi-barcelona',
   'name': 'Sensi',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/ndqinTlySy6slhGGeykyww/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/sensi-barcelona?adjust_creative=QyGIBXO3bewZz1kNqGAwCA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=QyGIBXO3bewZz1kNqGAwCA',
   'review_count': 402,
   'categories': [{'alias': 'tapasmallplates', 'title': 'Tapas/Small Plates'},
    {'alias': 'tapas', 'title': 'Tapas Bars'},
    {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}],
   'rating': 4.5,
   'coordinates': {'latitude': 41.38035, 'longitude': 2.17997},
   'transactions': [],
   'price': '€€',
   'location': {'address1': 'Carrer Ample, 26',
    'address2': '',
    'address3': '',
    'city': 'Barcelona',
    'zip_code': '08002',
    'country': 'ES',
    'state': 'B',
    'display_address': ['Carrer Ample, 26', '08002 Barcelona', 'Spain']},
   'phone': '+3493295

In [40]:
# Retreiving info about first business within the list
print(response_data['businesses'])

[{'id': 'Cn1geHmwrvKu1qQISgQKgA', 'alias': 'sensi-barcelona', 'name': 'Sensi', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/ndqinTlySy6slhGGeykyww/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/sensi-barcelona?adjust_creative=QyGIBXO3bewZz1kNqGAwCA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=QyGIBXO3bewZz1kNqGAwCA', 'review_count': 402, 'categories': [{'alias': 'tapasmallplates', 'title': 'Tapas/Small Plates'}, {'alias': 'tapas', 'title': 'Tapas Bars'}, {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}], 'rating': 4.5, 'coordinates': {'latitude': 41.38035, 'longitude': 2.17997}, 'transactions': [], 'price': '€€', 'location': {'address1': 'Carrer Ample, 26', 'address2': '', 'address3': '', 'city': 'Barcelona', 'zip_code': '08002', 'country': 'ES', 'state': 'B', 'display_address': ['Carrer Ample, 26', '08002 Barcelona', 'Spain']}, 'phone': '+34932956588', 'display_phone': '+34 932 95 65 88', 'distance': 1237.3374168059947}, {'id': 'Fb7Ni

In [41]:
## Retrieving the first business within the list 
response_data['businesses'][0]

{'id': 'Cn1geHmwrvKu1qQISgQKgA',
 'alias': 'sensi-barcelona',
 'name': 'Sensi',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/ndqinTlySy6slhGGeykyww/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/sensi-barcelona?adjust_creative=QyGIBXO3bewZz1kNqGAwCA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=QyGIBXO3bewZz1kNqGAwCA',
 'review_count': 402,
 'categories': [{'alias': 'tapasmallplates', 'title': 'Tapas/Small Plates'},
  {'alias': 'tapas', 'title': 'Tapas Bars'},
  {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}],
 'rating': 4.5,
 'coordinates': {'latitude': 41.38035, 'longitude': 2.17997},
 'transactions': [],
 'price': '€€',
 'location': {'address1': 'Carrer Ample, 26',
  'address2': '',
  'address3': '',
  'city': 'Barcelona',
  'zip_code': '08002',
  'country': 'ES',
  'state': 'B',
  'display_address': ['Carrer Ample, 26', '08002 Barcelona', 'Spain']},
 'phone': '+34932956588',
 'display_phone': '+34 932 95 65 88',
 'distance': 12

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [42]:
# Creating list with desired categories from Yelp
dining_cat_yelp = ['breweries', 'restaurants', 'pubs']

In [45]:
# Creating function and making request to yelp for each bike station
def get_poi_yelp(latitude, longitude, radius, API_KEY, categories):
    url = f'https://api.yelp.com/v3/businesses/search?latitude={latitude}&longitude={longitude}&radius={radius}&categories={categories}&sort_by=best_match'
    
    # Create dictionary for headers
    headers = {'Accept': 'application/json'}
    headers['Authorization'] = 'Bearer %s' % API_KEY
    response = requests.get(url, headers=headers)
    data = response.json()  
    data['station_id']= 'dedcd93fbe85671e73c79baf58bd7fed' ##this is the station id for "C/ CARDENAL TEDESCHINI"
    return data['businesses']

In [46]:
res_2 = get_poi_yelp(41.4253643, 2.1852073, 1000, API_KEY, categories=','.join(dining_cat_yelp))

In [47]:
## printing above results for one POI
res_2

[{'id': 'oD9g3jw6UoebuiI3GtSGww',
  'alias': 'tasca-i-vins-barcelona-7',
  'name': 'Tasca I Vins',
  'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/xXbBbalIAEpKg_KJt5Tlag/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/tasca-i-vins-barcelona-7?adjust_creative=QyGIBXO3bewZz1kNqGAwCA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=QyGIBXO3bewZz1kNqGAwCA',
  'review_count': 10,
  'categories': [{'alias': 'spanish', 'title': 'Spanish'},
   {'alias': 'tapas', 'title': 'Tapas Bars'}],
  'rating': 4.0,
  'coordinates': {'latitude': 41.4255799, 'longitude': 2.18318},
  'transactions': [],
  'price': '€',
  'location': {'address1': 'Calle Cardenal Tedeschini, 32',
   'address2': None,
   'address3': None,
   'city': 'Barcelona',
   'zip_code': '08027',
   'country': 'ES',
   'state': 'B',
   'display_address': ['Calle Cardenal Tedeschini, 32',
    '08027 Barcelona',
    'Spain']},
  'phone': '+34934082739',
  'display_phone': '+34 934 08 27 39',
  '

In [48]:
## Exploring above result of single POI from test call
print(res_2[0].keys(), '\n')
res_2['businesses'][0]

dict_keys(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'categories', 'rating', 'coordinates', 'transactions', 'price', 'location', 'phone', 'display_phone', 'distance']) 



TypeError: list indices must be integers or slices, not str

In [49]:
response_data['businesses'][0]

{'id': 'Cn1geHmwrvKu1qQISgQKgA',
 'alias': 'sensi-barcelona',
 'name': 'Sensi',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/ndqinTlySy6slhGGeykyww/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/sensi-barcelona?adjust_creative=QyGIBXO3bewZz1kNqGAwCA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=QyGIBXO3bewZz1kNqGAwCA',
 'review_count': 402,
 'categories': [{'alias': 'tapasmallplates', 'title': 'Tapas/Small Plates'},
  {'alias': 'tapas', 'title': 'Tapas Bars'},
  {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}],
 'rating': 4.5,
 'coordinates': {'latitude': 41.38035, 'longitude': 2.17997},
 'transactions': [],
 'price': '€€',
 'location': {'address1': 'Carrer Ample, 26',
  'address2': '',
  'address3': '',
  'city': 'Barcelona',
  'zip_code': '08002',
  'country': 'ES',
  'state': 'B',
  'display_address': ['Carrer Ample, 26', '08002 Barcelona', 'Spain']},
 'phone': '+34932956588',
 'display_phone': '+34 932 95 65 88',
 'distance': 12

In [50]:
## Loading the dataframe of bike stations that contain lat/long 
bike_stations_data2 = pd.read_csv('../data/bike_stations_barcelona.csv', index_col=0)

In [51]:
## Creating empty dataframe for yelp 
yelp_df = pd.DataFrame()

In [52]:
print(yelp_df.shape, '\n')
print(yelp_df.info())

(0, 0) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame
None


In [53]:
# Set the maximum number of API calls
max_api_calls = 40

# Counter for the number of API calls made
api_calls_count = 0

# Loop through the bike_stations_data2 dataframe to extract the lat/long/id of each row and make a new request to Yelp API
for index, row in bike_stations_data2.iterrows():
    # Make the http request using row attributes
    poi_yelp = get_poi_yelp(row['latitude'], row['longitude'], 1000, API_KEY, categories=','.join(dining_cat_yelp))
    
    # Check if the limit of API calls has been maximized 
    if api_calls_count >= max_api_calls:
        break

    # Turn the response into a dataframe
    new_df = pd.json_normalize(poi_yelp)
    # Add a column with the corresponding bike station id
    new_df['station_id'] = row['id'] 

    # Add the new dataframe to yelp_df
    yelp_df = pd.concat([yelp_df, new_df]) 

    # Increment the API calls counter
    api_calls_count += 1

In [54]:
print(yelp_df.shape, '\n')
print(yelp_df.info())

(792, 25) 

<class 'pandas.core.frame.DataFrame'>
Index: 792 entries, 0 to 19
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        792 non-null    object 
 1   alias                     792 non-null    object 
 2   name                      792 non-null    object 
 3   image_url                 792 non-null    object 
 4   is_closed                 792 non-null    bool   
 5   url                       792 non-null    object 
 6   review_count              792 non-null    int64  
 7   categories                792 non-null    object 
 8   rating                    792 non-null    float64
 9   transactions              792 non-null    object 
 10  price                     642 non-null    object 
 11  phone                     792 non-null    object 
 12  display_phone             792 non-null    object 
 13  distance                  792 non-null    float64
 14  coor

In [55]:
# Drop unnecessary columns from dataframe
columns_to_drop_yelp = [
    'alias', 'image_url', 'is_closed', 'url', 'transactions', 'phone',
    'display_phone', 'coordinates.latitude',
    'coordinates.longitude', 'location.address1', 'location.address2',
    'location.address3', 'location.city', 'location.zip_code',
    'location.country', 'location.state','categories']

yelp_df = yelp_df.drop(columns=columns_to_drop_yelp)

# Rename columns
yelp_df = yelp_df.rename(columns={"id": "poi_id", "location.display_address": "full_address"})

In [56]:
print(yelp_df.shape, '\n')
print(yelp_df.info())

(792, 8) 

<class 'pandas.core.frame.DataFrame'>
Index: 792 entries, 0 to 19
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   poi_id        792 non-null    object 
 1   name          792 non-null    object 
 2   review_count  792 non-null    int64  
 3   rating        792 non-null    float64
 4   price         642 non-null    object 
 5   distance      792 non-null    float64
 6   full_address  792 non-null    object 
 7   station_id    792 non-null    object 
dtypes: float64(2), int64(1), object(5)
memory usage: 55.7+ KB
None


In [57]:
## Printing dataframe to return first n rows 
print(yelp_df.head())

                   poi_id                          name  review_count  rating  \
0  oD9g3jw6UoebuiI3GtSGww                  Tasca I Vins            10     4.0   
1  J1vHLZvSRfkNpy2x8OuVaQ                    Torre Rosa            21     4.0   
2  iRrsa0bQbTtHzBjsHIprqQ             Beering Barcelona            10     4.5   
3  UdRmo2dstOYd29Ihs5vI8w  La Vermuteria de Sant Andreu             4     4.0   
4  e6bRub-8J0j5PDm55UNASA                     Toca Teca             3     4.5   

  price    distance                                       full_address  \
0     €  191.557179  [Calle Cardenal Tedeschini, 32, 08027 Barcelon...   
1    €€  501.236274  [Carrer de Francesc Tàrrega, 22, 08027 Barcelo...   
2     €  828.625543  [Carrer de las Navas de Tolosa, 355, 08041 Bar...   
3    €€  508.573281  [Rambla de Fabra I Puig, 51, 08030 Barcelona, ...   
4     €  395.734907    [Carrer Garcilaso, 172, 08027 Barcelona, Spain]   

                         station_id  
0  dedcd93fbe85671e73c79baf58b

In [59]:
## Dropping price and review count columns from dataframe
columns_to_drop_yelp = ['review_count', 'price']
yelp_df = yelp_df.drop(columns=columns_to_drop_yelp)

In [60]:
# Displaying dataframe with columns that have been removed 
yelp_df.head()

,poi_id,name,rating,distance,full_address,station_id
0,oD9g3jw6UoebuiI3GtSGww,Tasca I Vins,4.0,191.557179,"[Calle Cardenal Tedeschini, 32, 08027 Barcelon...",dedcd93fbe85671e73c79baf58bd7fed
1,J1vHLZvSRfkNpy2x8OuVaQ,Torre Rosa,4.0,501.236274,"[Carrer de Francesc Tàrrega, 22, 08027 Barcelo...",dedcd93fbe85671e73c79baf58bd7fed
2,iRrsa0bQbTtHzBjsHIprqQ,Beering Barcelona,4.5,828.625543,"[Carrer de las Navas de Tolosa, 355, 08041 Bar...",dedcd93fbe85671e73c79baf58bd7fed
3,UdRmo2dstOYd29Ihs5vI8w,La Vermuteria de Sant Andreu,4.0,508.573281,"[Rambla de Fabra I Puig, 51, 08030 Barcelona, ...",dedcd93fbe85671e73c79baf58bd7fed
4,e6bRub-8J0j5PDm55UNASA,Toca Teca,4.5,395.734907,"[Carrer Garcilaso, 172, 08027 Barcelona, Spain]",dedcd93fbe85671e73c79baf58bd7fed


In [68]:
# Assuming your DataFrame is named df
businesses_col = yelp_df['distance']

# Display the 'distance' column of above output 
print(businesses_col)

0      191.557179
1      501.236274
2      828.625543
3      508.573281
4      395.734907
         ...     
15     906.546334
16     671.128365
17     310.854028
18    1146.440757
19     559.148884
Name: distance, Length: 792, dtype: float64


In [69]:
yelp_df.head()['distance']

0    191.557179
1    501.236274
2    828.625543
3    508.573281
4    395.734907
Name: distance, dtype: float64

In [70]:
## Saving this data to use within another notebook 
yelp_df.to_csv('../data/bike_stations_barcelona_poi_yelp.csv')

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

In [78]:
## Printing Foursquare API Results 
print('foursquare results')
foursquare_df.head()

foursquare results


,poi_fsq_id,poi_name,category_id,category_name,poi_distance,poi_address,station_id
0,4e18b61d18a8166f738f1ed0,Mesón Pahi,13026,BBQ Joint,215,Calle Torroella de Montgrí 7,dedcd93fbe85671e73c79baf58bd7fed
1,4e18b61d18a8166f738f1ed0,Mesón Pahi,13338,Seafood Restaurant,215,Calle Torroella de Montgrí 7,dedcd93fbe85671e73c79baf58bd7fed
2,4e18b61d18a8166f738f1ed0,Mesón Pahi,13345,Spanish Restaurant,215,Calle Torroella de Montgrí 7,dedcd93fbe85671e73c79baf58bd7fed
3,590896ec5d891b372346f974,Cyclic Beer Farm,13029,Brewery,344,"Carrer de Coll, 37",dedcd93fbe85671e73c79baf58bd7fed
4,590896ec5d891b372346f974,Cyclic Beer Farm,13065,Restaurant,344,"Carrer de Coll, 37",dedcd93fbe85671e73c79baf58bd7fed


In [79]:
## Printing Yelp API results
print('Yelp results')
yelp_df.head()

Yelp results


,poi_id,name,rating,distance,full_address,station_id
0,oD9g3jw6UoebuiI3GtSGww,Tasca I Vins,4.0,191.557179,"[Calle Cardenal Tedeschini, 32, 08027 Barcelon...",dedcd93fbe85671e73c79baf58bd7fed
1,J1vHLZvSRfkNpy2x8OuVaQ,Torre Rosa,4.0,501.236274,"[Carrer de Francesc Tàrrega, 22, 08027 Barcelo...",dedcd93fbe85671e73c79baf58bd7fed
2,iRrsa0bQbTtHzBjsHIprqQ,Beering Barcelona,4.5,828.625543,"[Carrer de las Navas de Tolosa, 355, 08041 Bar...",dedcd93fbe85671e73c79baf58bd7fed
3,UdRmo2dstOYd29Ihs5vI8w,La Vermuteria de Sant Andreu,4.0,508.573281,"[Rambla de Fabra I Puig, 51, 08030 Barcelona, ...",dedcd93fbe85671e73c79baf58bd7fed
4,e6bRub-8J0j5PDm55UNASA,Toca Teca,4.5,395.734907,"[Carrer Garcilaso, 172, 08027 Barcelona, Spain]",dedcd93fbe85671e73c79baf58bd7fed


Get the top 10 restaurants according to their rating

In [83]:
# Making copy of the yelp dataframe with poi only
yelp_unique_poi = yelp_df[['poi_id', 'name', 'rating', 'distance', 'full_address', 'station_id']].copy()

# Convert the 'full_address' column to strings
yelp_unique_poi['full_address'] = yelp_unique_poi['full_address'].astype(str)

# Drop any duplicates based on all columns
yelp_unique_poi.drop_duplicates(inplace=True, ignore_index=True)

# Display the result DataFrame
print(yelp_unique_poi.head())

                   poi_id                          name  rating    distance  \
0  oD9g3jw6UoebuiI3GtSGww                  Tasca I Vins     4.0  191.557179   
1  J1vHLZvSRfkNpy2x8OuVaQ                    Torre Rosa     4.0  501.236274   
2  iRrsa0bQbTtHzBjsHIprqQ             Beering Barcelona     4.5  828.625543   
3  UdRmo2dstOYd29Ihs5vI8w  La Vermuteria de Sant Andreu     4.0  508.573281   
4  e6bRub-8J0j5PDm55UNASA                     Toca Teca     4.5  395.734907   

                                        full_address  \
0  ['Calle Cardenal Tedeschini, 32', '08027 Barce...   
1  ['Carrer de Francesc Tàrrega, 22', '08027 Barc...   
2  ['Carrer de las Navas de Tolosa, 355', '08041 ...   
3  ['Rambla de Fabra I Puig, 51', '08030 Barcelon...   
4  ['Carrer Garcilaso, 172', '08027 Barcelona', '...   

                         station_id  
0  dedcd93fbe85671e73c79baf58bd7fed  
1  dedcd93fbe85671e73c79baf58bd7fed  
2  dedcd93fbe85671e73c79baf58bd7fed  
3  dedcd93fbe85671e73c79baf58bd7fed 

In [84]:
## Filtering above results by rating, name, and distance
yelp_unique_poi.sort_values(by=['name', 'rating', 'distance'], ascending=False).head(10)

,poi_id,name,rating,distance,full_address,station_id
77,beISuI2RsALUavVH_RgVpA,Ølgod,4.5,317.699441,"[""Carrer de l'Hospital, 74"", '08001 Barcelona'...",4a30020e6926a2ba39f88be0f6bba5cb
630,rOiwFoXOZTp1yxVLCyC7Dg,Öuzo,5.0,360.455378,"['Rambla de Fabra i Puig, 66', '08030 Barcelon...",a08c92d249cec30792553ab20536cf99
34,rOiwFoXOZTp1yxVLCyC7Dg,Öuzo,5.0,316.560302,"['Rambla de Fabra i Puig, 66', '08030 Barcelon...",5c10c2dba1c5a8e8d9dcbb251fd8eab7
438,rOiwFoXOZTp1yxVLCyC7Dg,Öuzo,5.0,137.129150,"['Rambla de Fabra i Puig, 66', '08030 Barcelon...",d838e012f31b57bf9dc347842df5d1d6
369,zeVaxJ3XnougQ1sTQemfJw,las Cuatro Efes,5.0,1271.147945,"['Calle Segimon 10', '08032 Barcelona', 'Spain']",c1e818c978c673efb4fb8da5e7e51791
254,BCVlmrwt7tlY9s2S0ZhDgw,la masia,3.0,520.446950,"['alts forns, 61 calle', '08032 Barcelona', 'S...",4adedd174616ff28ded24a9ad859660f
713,I-c4V9IEE9KZ4ZmhoA9rDQ,Zarautz,4.5,386.581873,"[""Carrer de l'Elisi, 13"", '08014 Barcelona', '...",2859e381ef181038d89fdc04ef067165
533,I-c4V9IEE9KZ4ZmhoA9rDQ,Zarautz,4.5,282.374477,"[""Carrer de l'Elisi, 13"", '08014 Barcelona', '...",bd53dacde784bdd4a6fbea912d0fff75
232,Zx7QKQZzW4qdd1wlIL7Chg,Yon Gua,5.0,113.471528,"['Calle Maresme 197', '08020 Barcelona', 'Spain']",f28ea408c98c269ec16dd1a110ba77cc
202,7OYBlCoI6c9SnDyOwojPTQ,Xorus,4.5,1172.516306,"['Rambla de Poblenou, 105', '08005 Barcelona',...",de0e8ba6e1c44e2b7c452a639777e19a
